In [74]:
from warnings import filterwarnings
filterwarnings('ignore')

import numpy as np
import pandas as pd

## API Bağlantısının Yapılması

In [2]:
import tweepy,codecs

In [3]:
# keys

In [4]:
consumer_key = 'GoZTyDDaHOQ1YreBZViTyDFTE'
consumer_secret = 'YxUt3pGFfkmCHz6Mbo77dLhPA1dCPilyDQXpXg8bz0e97Ssp7B'
access_token = '967443566648872961-Q8gso7kaTOF2fD9Lm9INadLKF2qCH9B'
access_token_secret = 'QYKILttsvXVqg9aerSGuTJGiCbk6AVLMJKnZJEd80qTaZ'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

## Takipçi Segmentasyonu
amacımız takipçilerimiz analiz edip takipçilerimizin tweet etki güçlerini ölçmek istiyoruz... <br>
bunun için : <br>
        
        takipçi sayısı * atılan tweet => momentumunu kullanacağız.

In [5]:
user = api.get_user(id = 'fatihportakal')

In [6]:
followers = user.followers()

In [7]:
import pandas as pd

def friends_df(takipçi):
    
    data = [[k.id,
             k.created_at,
             k.screen_name,
             k.location,
             k.followers_count,
             k.statuses_count,
             k.friends_count,
             k.favourites_count] for k in takipçi]
    
    
    columns = ['id','created_at','screen_name','location','followers_count','statuses_count','friends_count','favourites_count']
    
    return pd.DataFrame(data,columns=columns)

In [8]:
df = friends_df(followers)

In [9]:
df.head()

,id,created_at,screen_name,location,followers_count,statuses_count,friends_count,favourites_count
0,1424649659100848128,2021-08-09 08:32:35,UfukKo82023293,,0,0,56,0
1,1424655023682772993,2021-08-09 08:53:40,NcyZclk,,0,0,9,0
2,1424656116374441984,2021-08-09 08:57:42,bafmn1628499449,,0,1,5,0
3,1363936115355504647,2021-02-22 19:38:19,ZlGokcen,,12,37,35,67
4,1424640979055874053,2021-08-09 07:57:41,godsavemymind,,0,0,22,0


In [10]:
# screen_name => index

In [11]:
df.index = df.screen_name

#### segmentasyon feature seçimi

In [47]:
s_data = df[['followers_count','statuses_count']]

In [48]:
s_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, UfukKo82023293 to vrnce1628499151
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   followers_count  20 non-null     int64
 1   statuses_count   20 non-null     int64
dtypes: int64(2)
memory usage: 1.0+ KB


In [49]:
s_data.head()

,followers_count,statuses_count
screen_name,,
UfukKo82023293,0,0
NcyZclk,0,0
bafmn1628499449,0,1
ZlGokcen,12,37
godsavemymind,0,0


#### Process
* 0 olan verileri düzenlemek... <br>
* scaling işlemi...

****Not**** : Bu segmentasyon işlemleri normalde tehlikelidir... <br>
Böyle bir segmentasyon işlemi (a*b | scaled values) için çarpılan feature'ların ****ağırlıklarını <br>
eşit kabul etmemiz**** gerekir.

In [50]:
# followers_count * statuses_count 

In [51]:
# ilk önce verilerdeki uzaklığı ayarlamak için standartlaştırma yapalım...

In [52]:
s_data = s_data.apply(lambda x : (x - min(x)) / (max(x) - min(x)))

In [53]:
# 0 değerleri bu değeri etkisizleştirdiğinden 0 değerleri yerine 0.01 verelim...

In [54]:
s_data = s_data + 0.01

In [55]:
s_data.head()

,followers_count,statuses_count
screen_name,,
UfukKo82023293,0.010000,0.010000
NcyZclk,0.010000,0.010000
bafmn1628499449,0.010000,0.010554
ZlGokcen,0.055113,0.030499
godsavemymind,0.010000,0.010000


#### Skor Hesaplama (Momentum)

In [56]:
score = s_data.followers_count * s_data.statuses_count

In [57]:
score

screen_name
UfukKo82023293     0.000100
NcyZclk            0.000100
bafmn1628499449    0.000106
ZlGokcen           0.001681
godsavemymind      0.000100
farukka53009229    0.000100
madamceycey        1.020100
NginZzeynep        0.000100
YusufNurdoan3      0.000100
yeldirik1          0.000100
egela_es           0.000100
Gnl07167545        0.000145
WhiteSa42450934    0.000145
zlmxn1628499080    0.000100
SefaKar01781472    0.000100
YelizUzunay2       0.000213
Murat13531997      0.000344
Murat94123737      0.000100
McYirmak           0.000100
vrnce1628499151    0.000272
dtype: float64

#### Segmentasyon
takipçileri kümelere segmente etmek için 2 yöntem olabilir. <br>

* El ile (az gözlem olursa uygulanabilir. İlkeldir...)
* Unsupervised ML ile  

In [61]:
# el ile

In [69]:
score[score > score.median() + score.std() / len(score)].sort_values(ascending=False)

screen_name
madamceycey    1.0201
dtype: float64

In [70]:
s_data['skor'] =  score

In [71]:
s_data.head()

,followers_count,statuses_count,skor
screen_name,,,
UfukKo82023293,0.010000,0.010000,0.000100
NcyZclk,0.010000,0.010000,0.000100
bafmn1628499449,0.010000,0.010554,0.000106
ZlGokcen,0.055113,0.030499,0.001681
godsavemymind,0.010000,0.010000,0.000100


In [72]:
# koşulu sağlayan => A & sağlamayan => B

In [75]:
s_data['segment'] = np.where(s_data['skor'] >= score.median() + score.std() / len(score), 'A','B')

In [77]:
s_data.head()

,followers_count,statuses_count,skor,segment
screen_name,,,,
UfukKo82023293,0.010000,0.010000,0.000100,B
NcyZclk,0.010000,0.010000,0.000100,B
bafmn1628499449,0.010000,0.010554,0.000106,B
ZlGokcen,0.055113,0.030499,0.001681,B
godsavemymind,0.010000,0.010000,0.000100,B
